# Eye Open/Closed Detection Model Training


In [1]:
#imports
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import MobileNet_V2_Weights
from tqdm.notebook import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)


Device: cuda


In [2]:
# Paths (edit these to match your dataset)
data_dir = Path('data/MRL_Eye_Dataset')
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

In [3]:
#Data transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),#input size for mobilenetv2
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
#Data loaders
train_ds = datasets.ImageFolder(train_dir, transform=train_transforms)
test_ds = datasets.ImageFolder(test_dir, transform=test_transforms)

batch_size = 32 #according to system capability
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
print('Classes:', train_ds.classes)
class_to_idx = train_ds.class_to_idx
print('Class to idx mapping:', class_to_idx)


Classes: ['close eyes', 'open eyes']
Class to idx mapping: {'close eyes': 0, 'open eyes': 1}


## Model (Transfer Learning: MobileNetV2 backbone)
Using a pretrained backbone speeds up convergence and often performs better than training from scratch.

In [4]:
#importing mobilenetv2
#model = models.mobilenet_v2(pretrained=True) 
model = models.mobilenet_v2(weights=None)
#replacing classifier
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_features, 2)
)
model = model.to(device)

In [5]:
#loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

In [6]:
#training and validation functions
from copy import deepcopy
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for imgs, labels in tqdm(loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    for imgs, labels in tqdm(loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

#training loop
best_model_wts = deepcopy(model.state_dict())
best_acc = 0.0
num_epochs = 8

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    scheduler.step(val_loss)
    print(f'Epoch {epoch+1}/{num_epochs} - train_loss: {train_loss:.5f} acc: {train_acc:.5f} | val_loss: {val_loss:.5f} val_acc: {val_acc:.5f}')
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = deepcopy(model.state_dict())
#using best model weights according to validation accuracy
model.load_state_dict(best_model_wts)
print('Best val acc:', best_acc)


  0%|          | 0/2553 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch 1/8 - train_loss: 0.18239 acc: 0.92207 | val_loss: 0.15838 val_acc: 0.93981


  0%|          | 0/2553 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

...

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch 7/8 - train_loss: 0.03215 acc: 0.98887 | val_loss: 0.13868 val_acc: 0.95718


  0%|          | 0/2553 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch 8/8 - train_loss: 0.03071 acc: 0.98931 | val_loss: 0.42068 val_acc: 0.88706
Best val acc: 0.9739373254731617


In [7]:
#save the trained model
model_path = 'models/eye_detector_mobilenetv2_(MRL_dataset).pth'
torch.save({'model_state_dict': model.state_dict(),
            'class_to_idx': class_to_idx}, model_path)
print('Saved model to', model_path)


Saved model to models/eye_detector_mobilenetv2_(MRL_dataset).pth
